In [1]:
# ============================================================================
# Imports
# ============================================================================

import os
import pandas as pd
import sys
sys.path.append('text-autocomplete')
import requests
from src.data_utils import *
from src.eval_transformer_pipeline import TransformerEvaluator
from src.next_token_dataset import NextTokenDataset
from torch.utils.data import DataLoader
from src.lstm_model import create_model
from src.lstm_train import train
from src.eval_lstm import evaluate_lstm, print_examples
import torch

In [2]:
DATA_PATH = "dataset/tweets.txt"
URL = "https://code.s3.yandex.net/deep-learning/tweets.txt"

if not os.path.exists(DATA_PATH):
    print("Downloading dataset...")
    os.makedirs("dataset", exist_ok=True)
    r = requests.get(URL)
    with open(DATA_PATH, "wb") as f:
        f.write(r.content)

In [3]:
# ============================================================================
# Configuration
# ============================================================================

VOCAB_SIZE = 100_000
BATCH_SIZE = 1024
N_SAMPLES = 500_000  # use 500k tweets for faster experimentation

In [4]:
# ============================================================================
# Load and preprocess data
# ============================================================================

print("Loading raw texts...")
raw_texts = load_texts(DATA_PATH, n=N_SAMPLES)
print(f"Loaded {len(raw_texts)} raw texts")

print("\nCleaning texts...")
clean_texts = [clean(t) for t in raw_texts if len(clean(t).split()) > 1]
print(f"After cleaning: {len(clean_texts)} texts")

print("\nSplitting into train/val/test...")
train_texts, val_texts, test_texts = split_dataset(clean_texts)
print(f"Train: {len(train_texts)}, Val: {len(val_texts)}, Test: {len(test_texts)}")

print("\nBuilding vocabulary...")
stoi, itos = build_vocab(train_texts, VOCAB_SIZE)
print(f"Vocabulary size: {len(stoi)}")

Loading raw texts...
Loaded 500000 raw texts

Cleaning texts...
After cleaning: 495681 texts

Splitting into train/val/test...
Train: 396544, Val: 49568, Test: 49569

Building vocabulary...
Vocabulary size: 100000


In [5]:
# ============================================================================
# Create datasets and dataloaders
# ============================================================================

print("\nCreating datasets...")
train_ds = NextTokenDataset(train_texts, stoi)
val_ds = NextTokenDataset(val_texts, stoi)
print(f"Train dataset: {len(train_ds)} examples")
print(f"Val dataset: {len(val_ds)} examples")

print("\nCreating dataloaders...")
train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=lambda b: NextTokenDataset.collate_fn(b, stoi[PAD])
)
val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    collate_fn=lambda b: NextTokenDataset.collate_fn(b, stoi[PAD])
)

# check batch shape
x, y = next(iter(train_loader))
print(f"\nBatch shapes: x={x.shape}, y={y.shape}")


Creating datasets...
Train dataset: 396544 examples
Val dataset: 49568 examples

Creating dataloaders...

Batch shapes: x=torch.Size([1024, 31]), y=torch.Size([1024, 31])


In [6]:
# ============================================================================
# Inspect data
# ============================================================================

print("\n" + "="*70)
print("RAW TWEETS (first 10)")
print("="*70)
for i, text in enumerate(raw_texts[:10]):
    print(f"{i}: {text[:100]}")

print("\n" + "="*70)
print("CLEANED TWEETS (first 10)")
print("="*70)
for i, text in enumerate(clean_texts[:10]):
    print(f"{i}: {text[:100]}")

print("\n" + "="*70)
print("TOP 20 WORDS IN VOCABULARY")
print("="*70)
vocab_words = [w for w in stoi.keys() if w not in [PAD, UNK, EOS]][:20]
for i, word in enumerate(vocab_words):
    print(f"{i+1:2d}. '{word}' -> {stoi[word]}")

print("\n" + "="*70)
print("EXAMPLE FROM BATCH")
print("="*70)
x_sample = x[0]
y_sample = y[0]

# find actual length (excluding padding)
seq_len = (x_sample != stoi[PAD]).sum().item()

# decode tokens
x_words = [itos[int(idx)] for idx in x_sample[:seq_len] if int(idx) not in [stoi[PAD], stoi[EOS]]]
y_words = [itos[int(idx)] for idx in y_sample[:seq_len] if int(idx) not in [stoi[PAD], stoi[EOS]]]

print(f"Input sequence:  {' '.join(x_words)}")
print(f"Target sequence: {' '.join(y_words)}")
print(f"(notice target is shifted by 1 token)")


RAW TWEETS (first 10)
0: @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third D
1: is upset that he can't update his Facebook by texting it... and might cry as a result  School today 
2: @Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds
3: my whole body feels itchy and like its on fire
4: @nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all o
5: @Kwesidei not the whole crew
6: Need a hug
7: @LOLTrish hey  long time no see! Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you ?
8: @Tatiana_K nope they didn't have it
9: @twittera que me muera ?

CLEANED TWEETS (first 10)
0: a thats a bummer you shoulda got david carr of third day to do it d
1: is upset that he cant update his facebook by texting it and might cry as a result school today also 
2: i dived many times for the ball managed to save the rest go out of bounds
3: my whole body feels itch

In [7]:
# ============================================================================
# Test LSTM Model
# ============================================================================

model = create_model(
    vocab_size=len(stoi),
    embed_size=128,
    hidden_size=256,
    num_layers=2,
    dropout=0.1
)

# test forward pass
x_test, y_test = next(iter(train_loader))
logits = model(x_test)
print(f"Forward pass test: input {x_test.shape} -> logits {logits.shape}")

# test greedy generation
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

test_prefix = "i am"
test_tokens = tokenize(test_prefix, stoi)
print(f"\nTest generation:")
print(f"Prefix: '{test_prefix}'")

generated = model.generate_greedy(
    start_tokens=test_tokens,
    max_new_tokens=10,
    eos_idx=stoi[EOS],
    device=device
)

generated_text = ' '.join([itos[t] for t in generated])
print(f"Generated: '{generated_text}'")

Created LSTM model with 39,421,600 parameters
Forward pass test: input torch.Size([1024, 33]) -> logits torch.Size([1024, 33, 100000])

Test generation:
Prefix: 'i am'
Generated: 'i am receptionist receptionist empretty methquot methquot chruch visits manbits prieta ulumnya'


In [8]:
# ============================================================================
# Train LSTM Model
# ============================================================================

# training config
EPOCHS = 10
LR = 1e-3
print(f"Training on device: {device}")

# train model
train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    stoi=stoi,
    itos=itos,
    eos_idx=stoi[EOS],
    epochs=EPOCHS,
    lr=LR,
    device=device,
    print_examples_every=5
)

Training on device: cuda

TRAINING


Training: 100%|██████████| 388/388 [02:35<00:00,  2.50it/s]


Epoch  1/10 | Loss: 6.9267 | Val ROUGE-1: 0.3667


Training: 100%|██████████| 388/388 [02:30<00:00,  2.58it/s]


Epoch  2/10 | Loss: 6.4276 | Val ROUGE-1: 0.3611


...

Epoch  8/10 | Loss: 5.3882 | Val ROUGE-1: 0.3800


Training: 100%|██████████| 388/388 [02:31<00:00,  2.56it/s]


Epoch  9/10 | Loss: 5.2883 | Val ROUGE-1: 0.3917


Training: 100%|██████████| 388/388 [02:30<00:00,  2.57it/s]


Epoch 10/10 | Loss: 5.1973 | Val ROUGE-1: 0.3444

----------------------------------------------------------------------
Examples after epoch 10:
----------------------------------------------------------------------
Prefix:    see thats
Target:    why
Greedy:    a
Sampled:   when
----------------------------------------------------------------------

TRAINING COMPLETE


In [9]:
# ============================================================================
# Save Model
# ============================================================================

os.makedirs('text-autocomplete/models', exist_ok=True)

model_path = 'text-autocomplete/models/lstm_model.pt'

torch.save({
    'model_state_dict': model.state_dict(),
    'vocab_size': len(stoi),
    'embed_size': 128,
    'hidden_size': 256,
    'num_layers': 2,
    'dropout': 0.1,
    'stoi': stoi,
    'itos': itos,
    'config': {
        'vocab_size': VOCAB_SIZE,
        'batch_size': BATCH_SIZE,
        'epochs': EPOCHS,
        'lr': LR
    }
}, model_path)

print(f"Model saved to: {model_path}")

Model saved to: text-autocomplete/models/lstm_model.pt


In [10]:
# ============================================================================
# Evaluate LSTM Model 
# ============================================================================

print("\nEvaluating on validation set (GREEDY decoding)...")
val_scores = evaluate_lstm(
    model=model,
    data_loader=val_loader,
    itos=itos,
    eos_idx=stoi[EOS],
    pad_idx=stoi[PAD],
    device=device,
    num_samples=None,
    use_greedy=True  
)

print("\n" + "="*70)
print("VALIDATION ROUGE SCORES (GREEDY)")
print("="*70)
for metric, score in val_scores.items():
    print(f"{metric:12s}: {score:.4f}")
print("="*70)

print_examples(
    model=model,
    data_loader=val_loader,
    stoi=stoi,
    itos=itos,
    eos_idx=stoi[EOS],
    device=device,
    num_examples=10,
    show_both=True  # покажет и greedy, и sampled
)


Evaluating on validation set (GREEDY decoding)...

VALIDATION ROUGE SCORES (GREEDY)
rouge1      : 0.1166
rouge2      : 0.0215
rougeL      : 0.1163
rougeLsum   : 0.1163

GENERATION EXAMPLES

Example 1:
Prefix:    not really its gone too far for me to even get whats going on that only makes it
Target:    even scarier so i dont watch it
Greedy:    worse
Sampled:   go too the day on the own

Example 2:
Prefix:    cant say i have that <unk>
Target:    the <unk> <unk>
Greedy:    
Sampled:   thingtoo

Example 3:
Prefix:    i wish you could go to the
Target:    dentist for me
Greedy:    beach
Sampled:   wedding show i

Example 4:
Prefix:    oh sooooo bored broke a string
Target:    practicing fml
Greedy:    of the
Sampled:   cousin will

Example 5:
Prefix:    i missed yall y did yall
Target:    go first smh
Greedy:    get to see
Sampled:   do that bit

Example 6:
Prefix:    why do some people hate me ox it
Target:    makes me sad
Greedy:    is so sad
Sampled:   sucks i want

Example 7:
Prefix

In [11]:
# ============================================================================
# Evaluate DistilGPT2
# ============================================================================

evaluator = TransformerEvaluator(model_name='distilgpt2', device=device)

# evaluate on validation set
print("\nEvaluating DistilGPT2...")
gpt_scores = evaluator.evaluate(
    texts=val_texts,
    prefix_ratio=0.75,
    num_samples=None,  
    max_new_tokens=20,
    temperature=0.8,
    top_k=50,
    top_p=0.95
)

print("\n" + "="*70)
print("DISTILGPT2 ROUGE SCORES")
print("="*70)
for metric, score in gpt_scores.items():
    print(f"{metric:12s}: {score:.4f}")
print("="*70)

# print examples
evaluator.print_examples(
    texts=val_texts,
    num_examples=10,
    prefix_ratio=0.75,
    max_new_tokens=20,
    temperature=0.8,
    top_k=50,
    top_p=0.95
)

Loading distilgpt2...


Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded on cuda
Model has 81,912,576 parameters

Evaluating DistilGPT2...

Evaluating on 49568 samples...


Evaluating: 100%|██████████| 49568/49568 [1:22:35<00:00, 10.00it/s]
The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



DISTILGPT2 ROUGE SCORES
rouge1      : 0.0339
rouge2      : 0.0019
rougeL      : 0.0326
rougeLsum   : 0.0328

GENERATION EXAMPLES (DistilGPT2)

Example 1:
Prefix:    not really its gone too far for me to even get whats going on that only makes it eve
Target:    n scarier so i dont watch it
Greedy:    .
Sampled:   . Just about anything that is not going to make it go well.

Example 2:
Prefix:    cant say i have that probyr the onl
Target:    ywho talksme
Greedy:    ays are not going to be able to get the money.
Sampled:   anes and my t-shirts are still going to be doing that.
If you are interested in

Example 3:
Prefix:    i wish you could go to the den
Target:    tist for me
Greedy:    of the city of Düsseldorf.
Sampled:   of the house where he was tortured and killed, and then there is the house where you are tortured

Example 4:
Prefix:    oh sooooo bored broke a string pr
Target:    acticing fml
Greedy:    ong.
I was so excited to see how much I could do. I was so excited to
Sampled

In [12]:
# ============================================================================
# Compare LSTM vs DistilGPT2
# ============================================================================

comparison = pd.DataFrame({
    'LSTM': [
        val_scores['rouge1'],
        val_scores['rouge2'],
        val_scores['rougeL'],
        val_scores['rougeLsum']
    ],
    'DistilGPT2': [
        gpt_scores['rouge1'],
        gpt_scores['rouge2'],
        gpt_scores['rougeL'],
        gpt_scores['rougeLsum']
    ]
}, index=['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-Lsum'])

print("\n" + "="*70)
print("MODEL COMPARISON")
print("="*70)
print(comparison)
print("="*70)

print("\nDistilGPT2 improvement over LSTM:")
for metric in comparison.index:
    lstm_score = comparison.loc[metric, 'LSTM']
    gpt_score = comparison.loc[metric, 'DistilGPT2']
    improvement = ((gpt_score - lstm_score) / lstm_score * 100) if lstm_score > 0 else 0
    print(f"{metric:12s}: {improvement:+.1f}%")


MODEL COMPARISON
                LSTM  DistilGPT2
ROUGE-1     0.116622    0.033932
ROUGE-2     0.021478    0.001943
ROUGE-L     0.116340    0.032631
ROUGE-Lsum  0.116340    0.032773

DistilGPT2 improvement over LSTM:
ROUGE-1     : -70.9%
ROUGE-2     : -91.0%
ROUGE-L     : -72.0%
ROUGE-Lsum  : -71.8%


In [13]:
# ============================================================================
# Final Evaluation on Test Set
# ============================================================================

print("\n" + "="*70)
print("ФИНАЛЬНАЯ ОЦЕНКА НА ТЕСТОВОЙ ВЫБОРКЕ")
print("="*70)

test_ds = NextTokenDataset(test_texts, stoi)
test_loader = DataLoader(
    test_ds, 
    batch_size=BATCH_SIZE,
    collate_fn=lambda b: NextTokenDataset.collate_fn(b, stoi[PAD])
)

print(f"\nTest dataset: {len(test_ds)} examples")

# Evaluate 
print("\nEvaluating LSTM on TEST set (GREEDY decoding)...")
test_scores = evaluate_lstm(
    model=model,
    data_loader=test_loader,
    itos=itos,
    eos_idx=stoi[EOS],
    pad_idx=stoi[PAD],
    device=device,
    num_samples=None,  
    use_greedy=True
)

print("\n" + "="*70)
print("TEST SET ROUGE SCORES (LSTM)")
print("="*70)
for metric, score in test_scores.items():
    print(f"{metric:12s}: {score:.4f}")
print("="*70)

# Val vs Test
print("\n" + "="*70)
print("VALIDATION vs TEST COMPARISON")
print("="*70)
print(f"{'Метрика':<15} {'Validation':<12} {'Test':<12} {'Разница':<12}")
print("-" * 70)
for metric in ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']:
    val_score = val_scores[metric]
    test_score = test_scores[metric]
    diff = test_score - val_score
    print(f"{metric:<15} {val_score:<12.4f} {test_score:<12.4f} {diff:+.4f}")
print("="*70)

# check overfitting
if test_scores['rouge1'] < val_scores['rouge1'] - 0.02:
    print("\nВНИМАНИЕ: Test score существенно ниже Validation - возможен overfitting")
elif test_scores['rouge1'] > val_scores['rouge1'] + 0.02:
    print("\nХОРОШО: Test score выше Validation - модель хорошо генерализует")
else:
    print("\nОТЛИЧНО: Test и Validation scores близки - нет overfitting")

print_examples(
    model=model,
    data_loader=test_loader,
    stoi=stoi,
    itos=itos,
    eos_idx=stoi[EOS],
    device=device,
    num_examples=10,
    show_both=True
)


ФИНАЛЬНАЯ ОЦЕНКА НА ТЕСТОВОЙ ВЫБОРКЕ

Test dataset: 49569 examples

Evaluating LSTM on TEST set (GREEDY decoding)...

TEST SET ROUGE SCORES (LSTM)
rouge1      : 0.1176
rouge2      : 0.0210
rougeL      : 0.1173
rougeLsum   : 0.1173

VALIDATION vs TEST COMPARISON
Метрика         Validation   Test         Разница     
----------------------------------------------------------------------
rouge1          0.1166       0.1176       +0.0009
rouge2          0.0215       0.0210       -0.0004
rougeL          0.1163       0.1173       +0.0009
rougeLsum       0.1163       0.1173       +0.0009

ОТЛИЧНО: Test и Validation scores близки - нет overfitting

GENERATION EXAMPLES

Example 1:
Prefix:    grounded through the week not allowed to
Target:    use the comp
Greedy:    be in the
Sampled:   go in a

Example 2:
Prefix:    just got home i think i broke
Target:    my <unk> cords
Greedy:    my <unk>
Sampled:   getting ready to

Example 3:
Prefix:    oh dear i think its broken again ill try to fix it c

In [14]:
# ============================================================================
# FINAL REPORT
# ============================================================================

print("\n" + "="*70)
print("ИТОГОВЫЙ ОТЧЁТ ПО ПРОЕКТУ")
print("="*70)

print("\n📊 РЕЗУЛЬТАТЫ LSTM МОДЕЛИ:")
print("-" * 70)
print(f"{'Dataset':<15} {'ROUGE-1':<12} {'ROUGE-2':<12} {'ROUGE-L':<12}")
print("-" * 70)
print(f"{'Validation':<15} {val_scores['rouge1']:<12.4f} {val_scores['rouge2']:<12.4f} {val_scores['rougeL']:<12.4f}")
print(f"{'Test':<15} {test_scores['rouge1']:<12.4f} {test_scores['rouge2']:<12.4f} {test_scores['rougeL']:<12.4f}")
print(f"{'Разница':<15} {test_scores['rouge1']-val_scores['rouge1']:<12.4f} {test_scores['rouge2']-val_scores['rouge2']:<12.4f} {test_scores['rougeL']-val_scores['rougeL']:<12.4f}")
print("-" * 70)

print("\n📈 СРАВНЕНИЕ LSTM vs DistilGPT2:")
print("-" * 70)
print(f"{'Модель':<15} {'ROUGE-1':<12} {'ROUGE-2':<12} {'Параметры':<15}")
print("-" * 70)
lstm_params = sum(p.numel() for p in model.parameters())
gpt_params = sum(p.numel() for p in evaluator.model.parameters())
print(f"{'LSTM':<15} {test_scores['rouge1']:<12.4f} {test_scores['rouge2']:<12.4f} {lstm_params:>12,}")
print(f"{'DistilGPT2':<15} {gpt_scores['rouge1']:<12.4f} {gpt_scores['rouge2']:<12.4f} {gpt_params:>12,}")
print(f"{'Преимущество':<15} {test_scores['rouge1']/gpt_scores['rouge1']:<12.1f}x {test_scores['rouge2']/gpt_scores['rouge2']:<12.1f}x {gpt_params/lstm_params:>12.1f}x меньше")
print("-" * 70)


ИТОГОВЫЙ ОТЧЁТ ПО ПРОЕКТУ

📊 РЕЗУЛЬТАТЫ LSTM МОДЕЛИ:
----------------------------------------------------------------------
Dataset         ROUGE-1      ROUGE-2      ROUGE-L     
----------------------------------------------------------------------
Validation      0.1166       0.0215       0.1163      
Test            0.1176       0.0210       0.1173      
Разница         0.0009       -0.0004      0.0009      
----------------------------------------------------------------------

📈 СРАВНЕНИЕ LSTM vs DistilGPT2:
----------------------------------------------------------------------
Модель          ROUGE-1      ROUGE-2      Параметры      
----------------------------------------------------------------------
LSTM            0.1176       0.0210         39,421,600
DistilGPT2      0.0339       0.0019         81,912,576
Преимущество    3.5         x 10.8        x          2.1x меньше
----------------------------------------------------------------------


### Итоговый вывод

Для задачи автодополнения коротких текстов при ограниченном
объёме данных более эффективным решением является
специализированная LSTM-модель.

Трансформеры могут показать лучшие результаты при
дополнительном дообучении или при работе с более длинными
и сложными текстами, однако в текущих условиях LSTM
оказалась предпочтительной по качеству и вычислительной
эффективности.
